In [ ]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots
import json
import chart_studio
import chart_studio.plotly as py

In [ ]:
# Generate graph data for books sharing the same users who rate like both books
import pandas as pd
df = pd.read_csv("ratings.csv")
b = pd.read_csv("books.csv")
#b = b.sample(20)
b = b[0:250]
top_books = b['book_id'].to_list()
#df = df[df['book_id'] in top_books]
df = df[df['rating'].isin([5])]
df = df[df['book_id'].isin(top_books)]
books = {}
book_list = []

df = df.reset_index()
for i in range(0,df.shape[0]):
    book = df.loc[i,'book_id']
    user = df.loc[i,'user_id']

    if book not in books:
        books[book] = []
    books[book].append(user)

for key, val in books.items():
    book_list.append((key,val))

ofile = open('out.csv',"w+")
for i in range(0,len(book_list)-1):
    for j in range(i+1,len(book_list)):
        b1 = book_list[i][0]
        b2 = book_list[j][0]
        common_users = list(set(book_list[i][1]) & set(book_list[j][1]))
        count = len(common_users)
        if count > 1100:
            line = str(b1) + ',' + str(b2) + '\n'#+ ',' + str(count) + '\n'
            ofile.write(line)
ofile.close()

In [ ]:
traces = []
b = pd.read_csv("books.csv")
dfs = b[0:13][::-1]
#dfs = dfs.sort_values('original_publication_year',axis=0,ascending=False)
traces.append(
    go.Bar(x=dfs['average_rating'], y=dfs['original_title'], orientation='h'
                    , showlegend=False, marker= dict ( 
                        color= '#7dcea0'
                    )
    )
)
annotations = [
    dict(
        x= (l-1813)/(2012-1813)*3+0.4, 
        y=n,
        text= str(int(l)), # Some conditional to define outliers
        showarrow=False,
        xanchor='center',  # Position of text relative to x axis (left/right/center)
        yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
        font={'color':'white'}
    ) for n, l in zip(dfs['original_title'], dfs['original_publication_year'])
]

top_book_chart = go.Figure(data=traces,
    layout={
        'title':"Most popular books on Goodreads", "yaxis_title": "", "xaxis_title": "Average ratings",
        "xaxis": { "fixedrange": True,'showgrid':False, 'visible': True
            },
        'yaxis': {"fixedrange": True,'nticks':30}, 'bargap': 0.2,
        'annotations': annotations, 'margin': {'l':160},
        'plot_bgcolor': '#f4f6f6'
        
    }
)
top_book_chart.show()
py.plot(top_book_chart, filename = 'top_goodreads_books', auto_open=True)

In [ ]:
traces = []
b = pd.read_csv("outputfiles/controversials.csv")
dfs = b[0:10][::-1]
#dfs = dfs.sort_values('original_publication_year',axis=0,ascending=False)
traces.append(
    go.Bar(x=dfs['variance'], y=dfs['title'], orientation='h'
                    , showlegend=False, marker= dict ( 
                        color='#7dcea0'
                    )
    )
)
annotations = [
    dict(
        x= 0.1, 
        y=n,
        text= n, # Some conditional to define outliers
        showarrow=False,
        xanchor='left',  # Position of text relative to x axis (left/right/center)
        yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
        font={'color':'black'}
    ) for n, l in zip(dfs['title'], dfs['variance'])
]

control_chart = go.Figure(data=traces,
    layout={
        'title':"Most controversial books", "yaxis_title": "", "xaxis_title": "Variance",
        "xaxis": { "fixedrange": True, 'showgrid':False, 'visible': True
            },
        'yaxis': {"fixedrange": True,'visible':False}, 'bargap': 0.2,
        'annotations': annotations, 'margin': {'l':50},
        'plot_bgcolor': '#f4f6f6'
        
    }
)
control_chart.show()
py.plot(control_chart, filename = 'top_controlversial_books', auto_open=True)

In [ ]:
import plotly.graph_objects as go
import networkx as nx
df = pd.read_csv("out.csv", names=['a','b'])
print(df.shape)
A = list(df["a"].unique())
B = list(df["b"].unique())
node_list = list(set(A+B))
G = nx.Graph()
for i in node_list:
    G.add_node(i)
for i,j in df.iterrows():
    G.add_edges_from([(j["a"],j["b"])])

pos = nx.spring_layout(G, k=0.8, iterations=15000)
for n, p in pos.items():
    G.nodes[n]['pos'] = p

In [ ]:
books = pd.read_csv("books.csv")
books = books.set_index('book_id')

In [ ]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.8,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    name="",
    hovertemplate = '%{text}',
    marker=dict(
        showscale=False,
        colorscale='RdBu',
        reversescale=True,
        color='#7dcea0',
        size=13,
        line=dict(width=1,color="black")))

node_trace2 = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    name="",
    hovertemplate = '%{text}',
    marker=dict(
        showscale=False,
        colorscale='RdBu',
        reversescale=True,
        color='#196f3d',
        size=14,
        line=dict(width=1)))

node_trace3 = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    name="",
    hovertemplate = '%{text}',
    marker=dict(
        showscale=False,
        colorscale='RdBu',
        reversescale=True,
        color='#D7DF01',
        size=18,
        line=dict(width=1)))

annotations = []

shown_titles = ['Emma','Life of Pi','Gone Girl','Romeo and Juliet','The Princess Bride','A Game of Thrones']


shown_titles = ['Jane Eyre','Pride and Prejudice',"Harry Potter and the Philosopher's Stone",'The Hunger Games', 'The Hobbit','The Two Towers','A Game of Thrones','The Great Gatsby','Twilight','A Clash of Kings','1984','Animal Farm',' The Fellowship of the Ring','To Kill a Mockingbird','The Kite Runner']

shown_titles = ['1984','Fahrenheit 451','To Kill a Mockingbird','Romeo and Juliet','Life of Pi','The Grapes of Wrath', 'Wuthering Heights', 'The Secret Garden']

classics = ['1984','To Kill a Mockingbird']
def return_title(title):
    if 'Harry Potter' in title:
        return 'Harry Potter'
    return title

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    title = books.loc[node,'original_title']
    author = books.loc[node,'authors']
    if "Harry Potter" in title:
        node_trace2['x'] += tuple([x])
        node_trace2['y'] += tuple([y])
        node_trace2['text'] += tuple(['Title: ' + title+ '<br>' + 'Author: '+author ])
    elif title in classics:
        print(title)
        node_trace3['x'] += tuple([x])
        node_trace3['y'] += tuple([y])
        node_trace3['text'] += tuple(['Title: ' + title+ '<br>' + 'Author: '+author ])
    else:
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['text'] += tuple(['Title: ' + title+ '<br>' + 'Author: '+author])        
    annotations.append(
        dict(
             x=x, 
             y=y+0.06,
             text= return_title(title)if title in shown_titles else '',
        showarrow=False,
        font={'size':13},
        textangle = -15,
        xanchor='center',  # Position of text relative to x axis (left/right/center)
        yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
        )
    )
    

In [ ]:
graph_top = go.Figure(data=[edge_trace, node_trace, node_trace2, node_trace3],
             layout=go.Layout(
                title='Book network graph <br><i>(hover for the title and author)<i>',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                annotations=annotations,
                margin=dict(b=20,l=50,r=50),
                plot_bgcolor="#f4f6f6",
                xaxis=dict(fixedrange=True,showgrid=False, zeroline=False, showticklabels=False, range=[-0.3,0.75]),
                yaxis=dict(fixedrange=True,showgrid=False, zeroline=False, showticklabels=False, range=[0.003,0.9])))
graph_top.show()
py.plot(graph_top, filename = 'graph_top_books2', auto_open=True)